In [1]:
import os
import shutil
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization

import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import itertools

pd. set_option('display.max_colwidth', None)
tf.get_logger().setLevel('ERROR')

In [2]:
# keep it same as what is used for training
REMOVE_STOP_WORDS = False
APPLY_CUSTOM_CLEANING = False

In [3]:
def remove_stopwords(string):
    word_list = [word.lower() for word in string.split()]
    stopwords_list = list(stopwords.words("english"))
    for word in word_list:
        if word in stopwords_list:
            word_list.remove(word)
    return " ".join(word_list)


def clean_text(text):
    """
    make text lowercase, remove text in square brackets,
    remove punctuation and remove words containing numbers.
    """
    text = str(text).lower()
    text = re.sub("\\n", " ", text)
    text = re.sub(r"https\s+|www.\s+", r"", text)
    text = re.sub(r"http\s+|www.\s+", r"", text)
    text = re.sub(r"\’", "'", text)
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"wont", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"cant", "can not", text)
    text = re.sub(r"don\'t", "do not", text)
    text = re.sub(r"dont", "do not", text)
    text = re.sub(r"n\’t", " not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\’d", " would", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    text = re.sub(r"\n", "", text)
    text = re.sub(r"\r", "", text)
    text = re.sub(r"\'", "", text)
    text = re.sub(r"\"", "", text)
    text = re.sub(r'[?|!|\'|"|#]', r"", text)
    text = re.sub(r"[.|,|)|(|\|/]", r" ", text)
    text = re.sub(r"[0-9]+", "digit", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub("\[.*?\]", "", text)
    text = re.sub(r"\s+", " ", text)
    if REMOVE_STOP_WORDS:
        text = remove_stopwords(text)
    return text

In [25]:
saved_model_path = './apple5_model'
class_names = {1: "neutral", 0: "positive", 2:"negative"}
# class_names = {0: "neutral", 1: "positive", 2:"negative"}

In [22]:
reloaded_model = tf.saved_model.load(saved_model_path)

In [26]:
def print_my_examples(inputs, results):
  result_for_printing = \
    [f'input: {inputs[i]:<30} : score: {results[i]}  : class: {class_names[np.argmax(results[i])]}'
                         for i in range(len(inputs))]
  print(*result_for_printing, sep='\n')
  print()


examples = [
    'this is such an amazing movie!',  # this is the same sentence tried earlier
    'The movie was great!',
    'The movie was meh.',
    'The movie was okish.',
    'The movie was terrible...',
    'how can I login to app store?'
]
if APPLY_CUSTOM_CLEANING:
    examples = [clean_text(text) for text in examples]
reloaded_results = reloaded_model(tf.constant(examples))

print('Results from the saved model:')
print_my_examples(examples, reloaded_results)

Results from the saved model:
input: this is such an amazing movie! : score: [0.00374166 0.99136215 0.00489621]  : class: neutral
input: The movie was great!           : score: [0.00444103 0.9867165  0.00884237]  : class: neutral
input: The movie was meh.             : score: [0.22289412 0.6971022  0.08000365]  : class: neutral
input: The movie was okish.           : score: [0.07919228 0.8682619  0.0525458 ]  : class: neutral
input: The movie was terrible...      : score: [0.00729853 0.01884586 0.9738556 ]  : class: negative
input: how can I login to app store?  : score: [0.89651954 0.02201089 0.08146952]  : class: positive

